In [ ]:
import os
import glob
from six.moves import cPickle
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib import colors as mpl_colors
%matplotlib inline

cmap = mpl_colors.LinearSegmentedColormap.from_list('Custom cmap', [(0.0, 0.0, 0.0), (0.5, 0.0, 0.0), (0.0, 0.5, 0.0), (0.5, 0.5, 0.0), (0.0, 0.0, 0.5), (0.5, 0.0, 0.5), (0.0, 0.5, 0.5), (0.5, 0.5, 0.5), (0.25, 0.0, 0.0), (0.75, 0.0, 0.0), (0.25, 0.5, 0.0), (0.75, 0.5, 0.0),
(0.25, 0.0, 0.5), (0.75, 0.0, 0.5), (0.25, 0.5, 0.5), (0.75, 0.5, 0.5), (0.0, 0.25, 0.0), (0.5, 0.25, 0.0), (0.0, 0.75, 0.0), (0.5, 0.75, 0.0), (0.0, 0.25, 0.5)], 21)
def compare(cimg_id):
    if not os.path.exists('sec-preds/{}.pkl'.format(cimg_id)) or not os.path.exists('gain-preds/{}.pkl'.format(cimg_id)): return
    fig = plt.figure(figsize=(20,20))
    ax1, ax2, ax3, ax4 = fig.add_subplot('141'), fig.add_subplot('142'), fig.add_subplot('143'), fig.add_subplot('144')
    ax1.imshow(Image.open("data/VOCdevkit/VOC2012/JPEGImages/{}.jpg".format(cimg_id)).resize((321,321), Image.ANTIALIAS))
    ax2.imshow(cPickle.load(open("sec-preds/{}.pkl".format(cimg_id), 'rb')), vmin=0, vmax=21, cmap=cmap)
    ax3.matshow(cPickle.load(open("gain-preds/{}.pkl".format(cimg_id), 'rb')), vmin=0, vmax=21, cmap=cmap)
    ax4.imshow(Image.open("data/VOCdevkit/VOC2012/SegmentationClassAug/{}.png".format(cimg_id)).resize((321,321), Image.ANTIALIAS), cmap='gray')
    ax1.set_title(cimg_id)
    ax2.set_title("sec")
    ax3.set_title("gain")
    ax4.set_title("gt_mask")
    ax1.set_axis_off()
    ax2.set_axis_off()
    ax3.set_axis_off()
    ax4.set_axis_off()
    plt.show()


for f in glob.glob("data/VOCdevkit/VOC2012/JPEGImages/*.jpg"): compare(f.split('/')[-1].split('.')[0])